In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# Читаем CSV в Pandas Dataframe
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
# Объединим тестовую и тренировочную выборку для наилучшего представления о значениях признаков
all_data = train_data.append(test_data, ignore_index=True)
# Посчитаем индексы для последующего разделения объединенной выборки на тренировочную и тестовую
train_idx = len(train_data)
test_idx = len(all_data) - len(test_data)

In [4]:
# Посмотрим на полученный датафрейм
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Посмотрим на описание датафрейма
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB


Имеем 1309 пассажиров: из них 891 относится к тренировочной выборке, для них определено поле Survived. В полях Имя, Пол, состав семьи, билет нет пропусков. Возраст не определен у 250 человек, попробуем восстановить его. Номер каюты определен менее чем у 30% пассажиров, это поле не дает нам никаких объективных и полезных знаний. 

In [6]:
# Посмотрим на выживаемость в зависимости от класса и пола
train_data.groupby(["Pclass", "Sex"])["Survived"].value_counts(normalize=True)

Pclass  Sex     Survived
1       female  1           0.968085
                0           0.031915
        male    0           0.631148
                1           0.368852
2       female  1           0.921053
                0           0.078947
        male    0           0.842593
                1           0.157407
3       female  0           0.500000
                1           0.500000
        male    0           0.864553
                1           0.135447
Name: Survived, dtype: float64

Видим, что выживаемость напрямую связана с полом и классмо пассажира. Выживаемость женщин во всех классах выше, чем мужчин.Также выживаеость тем выше, чем класс пассажира

Для восстановления данных о возрасте используем имена пассажиров. Имена имеют обращение Мистер, Миссис и т.д. С помощью такого обращения можно определить к какой возрастной группе относится человек. Это поможет нам заменить пропуски на медиану возраста по данной возрастной группе.

In [7]:
# Выделяем обращение из имени
all_data['NamePrefix'] = all_data["Name"].apply(lambda name: name.split(',')[1].split('.')[0].strip())

In [8]:
# Посмотрим на полученные обращения
all_data['NamePrefix'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer', 'Dona'], dtype=object)

На борту титаника были пассажиры разных стран. Попробуем привести все обращения к одному виду: например Mme французкий эквивалент обращения Mrs. Создадим словарь для преобразования эквивалентых обращений:

In [9]:
normalized_titles = {
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"
}

all_data["NamePrefix"] = all_data["NamePrefix"].map(normalized_titles)
all_data["NamePrefix"].value_counts()

Mr         757
Miss       262
Mrs        200
Master      61
Officer     23
Royalty      6
Name: NamePrefix, dtype: int64

Теперь мы имеем отдельный признак для каждого пассажира: его статус в обществе через обращение к нему. Теперь посчитаем медианный возраст для каждой группы пассажиров в зависимости от их класса. 

In [10]:
grouped = all_data.groupby(['Sex','Pclass', 'NamePrefix'])  
grouped["Age"].median()

Sex     Pclass  NamePrefix
female  1       Miss          30.0
                Mrs           45.0
                Officer       49.0
                Royalty       39.0
        2       Miss          20.0
                Mrs           30.0
        3       Miss          18.0
                Mrs           31.0
male    1       Master         6.0
                Mr            41.5
                Officer       52.0
                Royalty       40.0
        2       Master         2.0
                Mr            30.0
                Officer       41.5
        3       Master         6.0
                Mr            26.0
Name: Age, dtype: float64

In [11]:
all_data["Age"] = grouped["Age"].apply(lambda x: x.fillna(x.median()))

In [12]:
#Посмотрим на изменения в колонке Возраст
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  NamePrefix   1309 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 133.1+ KB


Теперь мы не имеем пропуск в поле возраст. Это должно помочь при анализе, так как возраст пассаижра является одним из главных показателей выживаемости.

In [13]:
# Заполним пропуск в графе Fare на медианное значение
all_data["Fare"] = all_data["Fare"].fillna(all_data["Fare"].median())

# Посмотрим на изменения
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1309 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1309 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
 12  NamePrefix   1309 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 133.1+ KB


In [14]:
# Синтезируем новый признак: размер семьи. Это можно сделать на основании числа родителей и числа братьев/сестер на борту
all_data['FamilySize'] = all_data["Parch"] + all_data["SibSp"] + 1

Большим семьям было тяжело получить место в спастельной лодке, в отличие от одиноких пассажиров или маленьких семей. Попробуем синтезировать новый признак и избавить от двух имеющихся(Parch и SibSp).

In [15]:
# Посмотрим на устройство получившего датафрейма
all_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NamePrefix,FamilySize
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,2
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs,2
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss,1
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs,2
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr,1


In [16]:
# Нужно закодировать пол пассаижров. Мужчинам будет соответсовать 0, женщин 1.
all_data["Sex"] = all_data["Sex"].map({"male": 0, "female":1})

In [17]:
#Отбросим признаки, которые не будут участвовать в классификации: обращение к имени(мы его использовали для определния возраста),
# имя, билет, порт посадки, 
# братья/сестры и родители(их мы синтезировали в отдельный признак FamilySize), номер каюты(мало у кого определен)
all_data.drop(['NamePrefix', 'Cabin', 'Embarked','SibSp','Parch', 'Name', 'Ticket'], axis=1, inplace=True)
all_data.head()

,PassengerId,Survived,Pclass,Sex,Age,Fare,FamilySize
0,1,0.0,3,0,22.0,7.2500,2
1,2,1.0,1,1,38.0,71.2833,2
2,3,1.0,3,1,26.0,7.9250,1
3,4,1.0,1,1,35.0,53.1000,2
4,5,0.0,3,0,35.0,8.0500,1


Датасет готов для обучения и тестирования, можно начинать разделять его на тренировочную и тестовую части

In [18]:
# Рзаделям исправленный датасет
train = all_data[ :train_idx]
test = all_data[test_idx: ]

# Создаем numpy array с тренировочной выборкой и целевую переменную
X = train.drop('Survived', axis=1).values 
y = train["Survived"].values
y = y.astype("int")

# Создаем numpy array для предсказания
X_test = test.drop('Survived', axis=1).values
X

array([[  1.    ,   3.    ,   0.    ,  22.    ,   7.25  ,   2.    ],
       [  2.    ,   1.    ,   1.    ,  38.    ,  71.2833,   2.    ],
       [  3.    ,   3.    ,   1.    ,  26.    ,   7.925 ,   1.    ],
       ...,
       [889.    ,   3.    ,   1.    ,  18.    ,  23.45  ,   4.    ],
       [890.    ,   1.    ,   0.    ,  26.    ,  30.    ,   1.    ],
       [891.    ,   3.    ,   0.    ,  32.    ,   7.75  ,   1.    ]])

In [19]:
# Попробуем перебрать некоторые парметры случайного леса: максимальную глубинку дерева, 
# количесто деревьев в лесу и гиперпарметры разделения внутренних вершин дерева
forrest_params = dict(     
    max_depth = [n for n in range(9, 15)],     
    min_samples_split = [n for n in range(4, 11)], 
    min_samples_leaf = [n for n in range(2, 5)],     
    n_estimators = [n for n in range(10, 20, 1)],
)

In [20]:
# создаем экемпляр Случайного леса
forrest = RandomForestClassifier()

In [21]:
# Создаем класс для подрбора параметров случайного леса и обучаем модель
forest_cv = GridSearchCV(estimator=forrest, param_grid=forrest_params, cv=5) 
forest_cv.fit(X, y)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 10, 11, 12, 13, 14],
                         'min_samples_leaf': [2, 3, 4],
                         'min_samples_split': [4, 5, 6, 7, 8, 9, 10],
                         'n_estimators': [10, 11, 12, 13, 14, 15, 16, 17, 18,
                                          19]})

In [22]:
print("Наилучший рузльтат: {}".format(forest_cv.best_score_))
print("Оптимальные параметры: {}".format(forest_cv.best_estimator_))

Наилучший рузльтат: 0.8462494507563869
Оптимальные параметры: RandomForestClassifier(max_depth=11, min_samples_leaf=4, min_samples_split=9,
                       n_estimators=14)


In [23]:
# Используем обученную модель для предсказаний
forrest_pred = forest_cv.predict(X_test)

In [24]:
# Формируем датафрейм с предксакзаниями
passengerId = test_data["PassengerId"]
result = pd.DataFrame({'PassengerId': passengerId, 'Survived': forrest_pred})

# сохранем результат в файл
result.to_csv('titanic.csv', index=False)